In [1]:
!pip install transformers  --quiet

In [36]:
import pandas as pd
df_train = pd.read_csv("/kaggle/input/random-insertion/RandomInsertionAugData.csv")
df_train = df_train.dropna()
df_test = pd.read_csv("/kaggle/input/back-transilations/Test_clean.csv")
df_test = df_test.dropna()
df_test.head()

,headline,label
0,लग्नाच्या दुसऱ्याच दिवशी नदीत बुडून नवरदेवाचा ...,2
1,"आराखडा रद्द, शिवसेनेकडून निर्णयाचं स्वागत",2
2,"ट्रकची बसला जोरदार धडक, 5 जणांचा मृत्यू",2
3,आॅस्ट्रेलियन ओपनमध्ये सानिया पराभूत,1
4,भूपेन हजारिका यांचा जीवनप्रवास,0


In [37]:
import numpy as np
num_of_rows = int(len(df_train) * 0.9)
values = df_train.to_numpy()
np.random.shuffle(values)
df_train = values[:num_of_rows] #indexes rows for training data
df_validation = values[num_of_rows:] #indexes rows for test data

df_train = pd.DataFrame(df_train,columns=['text','label'])
df_validation = pd.DataFrame(df_validation,columns=['text','label'])

df_test.rename(columns={'headline':'text'},inplace=True)

print(df_train.shape,df_validation.shape,df_test.shape)

(7277, 2) (809, 2) (1010, 2)


In [38]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

df_train_hg = Dataset(pa.Table.from_pandas(df_train))
df_validation_hg = Dataset(pa.Table.from_pandas(df_validation))
df_test_hg = Dataset(pa.Table.from_pandas(df_test))

In [39]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import numpy as np

checkpoint = "distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["text"] ,truncation=True)


df_train_tokenised = df_train_hg.map(tokenize_function, batched=True)
df_validation_tokenised = df_validation_hg.map(tokenize_function,batched=True)
df_test_tokenised = df_test_hg.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [40]:
tf_train_dataset = df_train_tokenised.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = df_validation_tokenised.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

tf_test_dataset = df_test_tokenised.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [41]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint,from_pt=True,num_labels = 3)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

# batch_size = 8
num_epochs = 10
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)

In [43]:
import tensorflow as tf

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

In [44]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
early_stopping_callback = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 2, patience = 5 )

In [ ]:
history = model.fit(tf_train_dataset,callbacks=early_stopping_callback, validation_data=tf_validation_dataset,epochs=10,verbose=2)

Epoch 1/10
909/909 - 151s - loss: 0.4031 - accuracy: 0.8805 - val_loss: 0.1694 - val_accuracy: 0.9604 - 151s/epoch - 166ms/step
Epoch 2/10
909/909 - 98s - loss: 0.1422 - accuracy: 0.9626 - val_loss: 0.1150 - val_accuracy: 0.9666 - 98s/epoch - 107ms/step
Epoch 3/10
909/909 - 98s - loss: 0.0729 - accuracy: 0.9807 - val_loss: 0.0917 - val_accuracy: 0.9802 - 98s/epoch - 108ms/step
Epoch 4/10
909/909 - 99s - loss: 0.0363 - accuracy: 0.9922 - val_loss: 0.0609 - val_accuracy: 0.9852 - 99s/epoch - 109ms/step
Epoch 5/10
909/909 - 97s - loss: 0.0220 - accuracy: 0.9953 - val_loss: 0.0619 - val_accuracy: 0.9913 - 97s/epoch - 107ms/step
Epoch 6/10
909/909 - 97s - loss: 0.0134 - accuracy: 0.9974 - val_loss: 0.0578 - val_accuracy: 0.9913 - 97s/epoch - 107ms/step
Epoch 7/10
909/909 - 97s - loss: 0.0079 - accuracy: 0.9985 - val_loss: 0.0550 - val_accuracy: 0.9938 - 97s/epoch - 106ms/step
Epoch 8/10
909/909 - 97s - loss: 0.0045 - accuracy: 0.9983 - val_loss: 0.0335 - val_accuracy: 0.9963 - 97s/epoch - 1

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], c = 'b', label = 'train loss')
plt.plot(history.history['val_loss'], c = 'r', label = 'validation loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], c = 'b', label = 'train accuracy')
plt.plot(history.history['val_accuracy'] , c = 'r', label = 'validation accuracy')
plt.legend(loc='lower right')
plt.show()

Extracting classification report and confusion matrix

In [ ]:
preds = model.predict(tf_test_dataset)["logits"]
class_preds = np.argmax(preds, axis=1)
print(preds.shape, class_preds.shape)

In [ ]:
y_test = df_test.label.values
y_pred = class_preds

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
# evaluate predictions
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
print(metrics.classification_report( y_test,y_pred))

In [33]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[244   4  28]
 [  9  90   7]
 [ 11   3 614]]


In [34]:
cm_matrix = pd.DataFrame(data = cm, columns = ['Actual Positive: 0', 'Actual Negative: 1'], index = ['Predict Positive: 0', 'Predict Negative: 1'])
sns.heatmap(cm_matrix, annot = True, fmt = 'd', cmap = 'YlGnBu')

ValueError: Shape of passed values is (3, 3), indices imply (2, 2)